In [3]:
import pdfplumber
import pandas as pd

filename = "pdfs/mvfodevarer.pdf"

### Code to draw on PDF

In [9]:
cells_found = []

with pdfplumber.open("pdfs/mvfodevarer_side12.pdf") as pdf:
    print(pdf.pages)
    page = pdf.pages[0]
    #TODO: 
    # 1. Ignore rects with height = 0? (or under certain threshold)
    # 2. Ignore nested rects
    im = page.to_image(resolution = 400)

    # for i in range(0,len(page.rects)):
    for i in range(0,20):
    #Selected rects
        rect = page.rects[i]
        # if rect['height'] < 1 or rect['width'] < 1: continue #1 Ignore small rectangles
        cells_found.append(rect)
        # print(f"Rect index: {i}")
        # print(f"Rect height: {rect['height']}")
        # print(f"Rect width: {rect['width']}")
    im = im.draw_rects(cells_found)
    im.show()

[<Page:1>]


### Table extraction code
Rectangle selection algorithm:
1. Ignore rects with height or width < 1
2. Ignore nested rects

In [165]:
def extract_coordinates(rect):
    return (rect['x0'], rect['x1'],rect['y0'],rect['y1'])

def convert_float_to_int(t):
    return tuple(int(x) for x in t)    

def is_nested(rect, other_rect):    
    rect_coords = extract_coordinates(rect)
    other_rect_coords = extract_coordinates(other_rect)
    rect_coords = convert_float_to_int(rect_coords) #Convert floating point to integers to avoid issues caused by rounding errors
    other_rect_coords = convert_float_to_int(other_rect_coords) #Convert floating point to integers to avoid issues caused by rounding errors
    (x0,x1,y0,y1) = rect_coords
    (x0_o,x1_o,y0_o,y1_o) = other_rect_coords

    return x0_o <= x0 and x1_o >= x1 and y0_o <= y0 and y1_o >= y1

def find_cells_on_page(page):
    selected_rects = []

    initial_j = 0

    for i in range(0,len(page.rects)):
        rect = page.rects[i]
        if rect['height'] < 1 or rect['width'] < 1: continue #1 Ignore small rectangles

        #Look for rectangles that might encapsulate current rectangle on page
        for j in range(initial_j,len(page.rects)):
            #print(f"(i,j): ({i},{j})")
            other_rect = page.rects[j]
            if j == i: #Assumption: rectangles with index > i will never encapsulate rectangle i.
                #print(f"added rect {i} to selected rects")
                selected_rects.append(rect)
                break
            elif is_nested(rect, other_rect):
                initial_j = j #Displace the starting point to search for rectangles that nest rectangle i.
                break
    
    return selected_rects

def set_header(table):
    print(table)
    
    table = table[1:] #Removing first item since it is irrelevant
    #table = [[replace_newlines(cell) for cell in row] for row in table] #Replace newline characters with whitespace #TODO
    headers = table[0] #Extracting headers    
    table = table[1:] #Removing headers from data    
    return pd.DataFrame(table, columns=headers)



def transform_table(df: pd.DataFrame):

    headers = df.columns.to_list()
    return df.melt(
            id_vars=["Madvare"],
            value_vars=headers[1:],
            var_name="Enhed",
            value_name="Konverteringsfaktor"
            )

#Tests whether table is ready to be transformed and further incorporated in to database
def is_table_ready_for_transformation(table : pd.DataFrame) -> bool:
    #No merged cells  
    #(Only contains numbers)
    pass
   


def extract_pdf_tables(filename : str) -> dict:

    tables = []

    with pdfplumber.open(filename) as pdf:
        # Iterate through each page
        for page in pdf.pages:

            #Find cells on page
            cells_found = find_cells_on_page(page) 
            
            # Extract tables from the page
            page_tables = page.extract_tables({
                "vertical_strategy": "explicit",
                "horizontal_strategy": "explicit",
                "explicit_vertical_lines": cells_found,
                "explicit_horizontal_lines": cells_found,
            })
    
            if page_tables:
                for table in page_tables:
                    if table:
                        
                        table = set_header(table)
                        
                        
                        #if is_table_ready_for_transformation:
                        tables.append(table)

                        # transformed_table = transform_table(table)

                        # print(curated_table.to_markdown())
                        # print()
            break # For testing
    
    return tables

tables = extract_pdf_tables(filename)
# for i in tables: 
#     print(i.to_markdown())
#     break

[['Brød', None, None, None], ['Madvare', 'Lille', 'Mellem', 'Stor'], ['Bagel', '', '90', ''], ['Bolle, almindelig / fuldkorn – bager/industri', '40', '60', '80'], ['Bolle, almindelig / fuldkorn – hjemmebagt', '50', '70', '120'], ['Burger/hamburger bolle, almindelig/fuldkorn', '', '60', '90'], ['Ciabattabolle, almindelig/fuldkorn', '', '70', ''], ['Crouton', '0,4', '', ''], ['Flute/baguette – langt', '', '150', '300'], ['Flute/baguette – mini/kuvert', '', '50', ''], ['Focacciabolle', '', '120', ''], ['Hvedebrød, skive industribrød', '1 cm=25', '1½ cm=30', '2cm=40'], ['Hvedebrød, skive - hjemmebagt', '1 cm=30', '1½ cm=45', '2 cm=60'], ['Knækbrød, almindelig/fuldkorn', '5', '12', '26'], ['Knækbrød -bager', '', '35', ''], ['Krydderboller', '', '60', ''], ['Kuvertbrød/ bolle', '', '30', ''], ['Naan', '', '120', ''], ['Pandekage, blinis', '10', '', ''], ['Pandekage, industri', '30', '60', ''], ['Pitabrød, almindelig', '60', '80', '90'], ['Pitabrød, fuldkorn', '', '80', ''], ['Pizzabolle', ''

In [ ]:
#Implementation for handling merged cells
for i, table in enumerate(tables):
    # if i in [9, 10, 11, 12, 13, 14, 35]: #Headers with merged cells
        print(f"i: {i}")
        
        # 1 Replace NoneTypes
        new_columns = []
        last_regular_header = ""
        merged_cell_counter = 1
        merged_header_found = False
        for i, column in enumerate(table.columns):
            if column is None:
                merged_header_found = True
                merged_cell_counter = merged_cell_counter + 1
                new_columns.append(last_regular_header)

                if i == len(table.columns)-1:
                    for j in range(i+1-merged_cell_counter,i+1): #TODO: This is not a good approach.
                        new_columns[j] = new_columns[j] + "\n" + table.iat[0,j] 
            else:
                if merged_cell_counter > 1:
                    for j in range(i-merged_cell_counter,i):
                        new_columns[j] = new_columns[j] + "\n" + table.iat[0,j] 
                    merged_cell_counter = 1
                last_regular_header = column
                new_columns.append(column)
        table.columns = new_columns

        if merged_header_found:
            table = table.drop(table.index[[0]])



        print(table.to_markdown())
        print()

i: 0
hep
hep
hep
hep
|    | Madvare                                       | Lille   | Mellem   | Stor    |
|---:|:----------------------------------------------|:--------|:---------|:--------|
|  0 | Bagel                                         |         | 90       |         |
|  1 | Bolle, almindelig / fuldkorn – bager/industri | 40      | 60       | 80      |
|  2 | Bolle, almindelig / fuldkorn – hjemmebagt     | 50      | 70       | 120     |
|  3 | Burger/hamburger bolle, almindelig/fuldkorn   |         | 60       | 90      |
|  4 | Ciabattabolle, almindelig/fuldkorn            |         | 70       |         |
|  5 | Crouton                                       | 0,4     |          |         |
|  6 | Flute/baguette – langt                        |         | 150      | 300     |
|  7 | Flute/baguette – mini/kuvert                  |         | 50       |         |
|  8 | Focacciabolle                                 |         | 120      |         |
|  9 | Hvedebrød, skive industrib

In [12]:
# #Test dataframe
data = {
  "calories": [420, 380, 390],
  "duration": [50, 40, 45]
}

#load data into a DataFrame object:
df = pd.DataFrame(data)

print(df)
print(df.loc[2:,])

   calories  duration
0       420        50
1       380        40
2       390        45
   calories  duration
2       390        45


In [71]:
myList = ["a", "b", "c"] 

myList[0] + "ten"

'aten'

### Transformation

In [ ]:
def transform_table(df: pd.DataFrame):

    headers = df.columns.to_list()
    return df.melt(
            id_vars=["Madvare"],
            value_vars=headers[1:],
            var_name="Enhed",
            value_name="Konverteringsfaktor"
            )